<a href="https://colab.research.google.com/github/ancka019/ComputationsMethods/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`:

In [ ]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number


@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__)
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [6]:
# ваш код
#Недостаток: поддержка конструкции match...case начиная с версии python 3.10

def add_one(n):
    return n + 1


def subtract_one(n):
    return n - 1


def power(x, n):
    if n == 0:
        return 1
    else:
        return x * power(x, n-1)


def divide(x, y):
    if y == 0:
        raise ValueError("Cannot divide by zero")
    else:
        return x / y


def test_operations():
    assert add_one(1) == 2
    assert subtract_one(2) == 1
    assert power(2, 3) == 8
    assert divide(4, 2) == 2
    assert divide(1, 0) == ValueError("Cannot divide by zero")


test_operations()

AssertionError: ignored

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [ ]:
# ваш код
import math

def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n-1)

def exp_series(x, terms):
    return sum(x**n / fact(n) for n in range(terms))

def cos_series(x, terms):
    return sum((-1)**n * x**(2*n) / fact(2*n) for n in range(terms))

def sin_series(x, terms):
    return sum((-1)**n * x**(2*n + 1) / fact(2*n + 1) for n in range(terms))

def log_series(x, terms):
    return sum((-1)**n * x**(-n-1) / (n-1) for n in range(2, terms+2))

# Example usage
print(exp_series(1, 100))
print(cos_series(0, 100))
print(sin_series(0, 100))
print(log_series(math.e, 100))


2.7182818284590455
1.0
0.0
0.04239535920745795


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

<ipython-input-11-c0f6ae60c6a3>:6: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  derivative(f, 2.)


22.0

In [ ]:
import numpy as np

def central_difference(f, x, h=1e-5):
    return (f(x + h) - f(x - h)) / (2 * h)

# Теперь, мы можем использовать эту функцию для проверки работы исходной функции
x = np.linspace(-np.pi, np.pi, 1000)
y = func(x)

# Взятие производной первого порядка с шагом h = 1e-5
dfdx = central_difference(f, x)

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках

Генерацию случайных функций можно осуществить, например, двумя путями.
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:

def f_2V(x: float, y: float) -> float:
    return 3*x*x - 2*y*y*y

def f_3V(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y

f_2V_diff = Dual.diff_2V(f_2V)
f_3V_diff = Dual.diff_3V(f_3V)
x = 10
y = 10
z = 10
print(f_2V_diff(1.5, 3.28))
f_3V_diff(x, y, z)